# Pyspark SQL

Window functions perform a calculation across rows that are related to the current row. 

## Creating and querying a SQL table in Spark

### Create a SQL table from a dataframe

In [1]:
# from urllib.request import urlretrieve
# url = "https://assets.datacamp.com/production/repositories/3937/datasets/a367f6f461f670a364ab2a59afc25bc2e3fab157/trainsched.txt"
# data = urlretrieve(url,"trainsched.txt")

In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "pyspark-shell")

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.read.csv("trainsched.txt", header=True)
df.createOrReplaceTempView("table1")
df.show()

+--------+-------------+----+
|train_id|      station|time|
+--------+-------------+----+
|     324|San Francisco|7:59|
|     324|  22nd Street|8:03|
|     324|     Millbrae|8:16|
|     324|    Hillsdale|8:24|
|     324| Redwood City|8:31|
|     324|    Palo Alto|8:37|
|     324|     San Jose|9:05|
|     217|       Gilroy|6:06|
|     217|   San Martin|6:15|
|     217|  Morgan Hill|6:21|
|     217| Blossom Hill|6:36|
|     217|      Capitol|6:42|
|     217|       Tamien|6:50|
|     217|     San Jose|6:59|
+--------+-------------+----+



### Determine the column names of a table

In [3]:
spark.sql("SHOW COLUMNS FROM table1").show()

spark.sql("SELECT * FROM table1 LIMIT 0").show()

spark.sql("DESCRIBE table1").show()

+--------+
|col_name|
+--------+
|train_id|
| station|
|    time|
+--------+

+--------+-------+----+
|train_id|station|time|
+--------+-------+----+
+--------+-------+----+

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|train_id|   string|   null|
| station|   string|   null|
|    time|   string|   null|
+--------+---------+-------+



## Window function SQL

A window function operates on a set of rows and retuns a value for each row in the set and value can depend on other rows in the set. 

OVER clause designates this query as a window function query and must contain an ORDER BY clause that tells it how to sequence the rows.

### Running sums using window function SQL

In [4]:
query = """SELECT *, LEAD(time, 1) OVER (PARTITION BY train_id ORDER BY time) AS time_next
FROM table1 """
df =spark.sql(query)
df.show()

+--------+-------------+----+---------+
|train_id|      station|time|time_next|
+--------+-------------+----+---------+
|     217|       Gilroy|6:06|     6:15|
|     217|   San Martin|6:15|     6:21|
|     217|  Morgan Hill|6:21|     6:36|
|     217| Blossom Hill|6:36|     6:42|
|     217|      Capitol|6:42|     6:50|
|     217|       Tamien|6:50|     6:59|
|     217|     San Jose|6:59|     null|
|     324|San Francisco|7:59|     8:03|
|     324|  22nd Street|8:03|     8:16|
|     324|     Millbrae|8:16|     8:24|
|     324|    Hillsdale|8:24|     8:31|
|     324| Redwood City|8:31|     8:37|
|     324|    Palo Alto|8:37|     9:05|
|     324|     San Jose|9:05|     null|
+--------+-------------+----+---------+



In [5]:
df = df.withColumn("time_timestamp", df.time.cast("timestamp"))

df = df.withColumn("time_next_timestamp", df.time_next.cast("timestamp"))
df.show()

+--------+-------------+----+---------+-------------------+-------------------+
|train_id|      station|time|time_next|     time_timestamp|time_next_timestamp|
+--------+-------------+----+---------+-------------------+-------------------+
|     217|       Gilroy|6:06|     6:15|2021-09-19 06:06:00|2021-09-19 06:15:00|
|     217|   San Martin|6:15|     6:21|2021-09-19 06:15:00|2021-09-19 06:21:00|
|     217|  Morgan Hill|6:21|     6:36|2021-09-19 06:21:00|2021-09-19 06:36:00|
|     217| Blossom Hill|6:36|     6:42|2021-09-19 06:36:00|2021-09-19 06:42:00|
|     217|      Capitol|6:42|     6:50|2021-09-19 06:42:00|2021-09-19 06:50:00|
|     217|       Tamien|6:50|     6:59|2021-09-19 06:50:00|2021-09-19 06:59:00|
|     217|     San Jose|6:59|     null|2021-09-19 06:59:00|               null|
|     324|San Francisco|7:59|     8:03|2021-09-19 07:59:00|2021-09-19 08:03:00|
|     324|  22nd Street|8:03|     8:16|2021-09-19 08:03:00|2021-09-19 08:16:00|
|     324|     Millbrae|8:16|     8:24|2

In [6]:
from pyspark.sql.functions import unix_timestamp
df = df.withColumn("diff", (unix_timestamp(df.time_next_timestamp)-unix_timestamp(df.time_timestamp))/60)
df.select("train_id", "station", "time","diff").show()
df.createOrReplaceTempView("table1")

+--------+-------------+----+----+
|train_id|      station|time|diff|
+--------+-------------+----+----+
|     217|       Gilroy|6:06| 9.0|
|     217|   San Martin|6:15| 6.0|
|     217|  Morgan Hill|6:21|15.0|
|     217| Blossom Hill|6:36| 6.0|
|     217|      Capitol|6:42| 8.0|
|     217|       Tamien|6:50| 9.0|
|     217|     San Jose|6:59|null|
|     324|San Francisco|7:59| 4.0|
|     324|  22nd Street|8:03|13.0|
|     324|     Millbrae|8:16| 8.0|
|     324|    Hillsdale|8:24| 7.0|
|     324| Redwood City|8:31| 6.0|
|     324|    Palo Alto|8:37|28.0|
|     324|     San Jose|9:05|null|
+--------+-------------+----+----+



In [14]:
query = """SELECT train_id, station, time, diff, 
SUM(diff) OVER(PARTITION BY train_id ORDER BY time) AS running_total FROM table1"""

spark.sql(query).show()


+--------+-------------+----+----+-------------+
|train_id|      station|time|diff|running_total|
+--------+-------------+----+----+-------------+
|     217|       Gilroy|6:06| 9.0|          9.0|
|     217|   San Martin|6:15| 6.0|         15.0|
|     217|  Morgan Hill|6:21|15.0|         30.0|
|     217| Blossom Hill|6:36| 6.0|         36.0|
|     217|      Capitol|6:42| 8.0|         44.0|
|     217|       Tamien|6:50| 9.0|         53.0|
|     217|     San Jose|6:59|null|         53.0|
|     324|San Francisco|7:59| 4.0|          4.0|
|     324|  22nd Street|8:03|13.0|         17.0|
|     324|     Millbrae|8:16| 8.0|         25.0|
|     324|    Hillsdale|8:24| 7.0|         32.0|
|     324| Redwood City|8:31| 6.0|         38.0|
|     324|    Palo Alto|8:37|28.0|         66.0|
|     324|     San Jose|9:05|null|         66.0|
+--------+-------------+----+----+-------------+



## Dot notation and SQL

There is typically a dot notation equivalent of every SQL caluse including window functions.

* df.select("train_id", "station")
* df.select(df.train_id, df.station)
* df.select(col("train_id"), col("station"))


* df.select("train_id", "station").withColumnRenamed("train_id", "train") 
* df.select(col("train_id").alias("train"), "station")


* spark.sql("SELECT train_id AS train, station FROM schedule LIMIT 5")
* df.select(col("train_id").alias("train"), "station").limit(5).show()

Window functions can also be done in either SQL or dot notation.

* spark.sql("SELECT *, ROW_NUMBER() OVER(PARTITION BY train_id ORDER BY time) AS id FROM schedule")
* df.withColumn("id", row_number().over(Window.partitionBy("train_id").orderBy("time")))


* window = Window.partitionBy("train_id").orderBy("time") 
    * dfx = df.withColumn("next",lead("time", 1).over(window))


### Aggregation, step by step

In [23]:
spark.sql("SELECT train_id, MIN(time) AS start FROM table1 GROUP BY train_id").show()
df.groupBy("train_id").agg({"time":"min"}).withColumnRenamed("min(time)", "start").show()

spark.sql("SELECT train_id, MIN(time), MAX(time) FROM table1 GROUP BY train_id").show()
result = df.groupBy("train_id").agg({"time":"min","time":"max"})
result.show()
print(result.columns)

+--------+-----+
|train_id|start|
+--------+-----+
|     217| 6:06|
|     324| 7:59|
+--------+-----+

+--------+-----+
|train_id|start|
+--------+-----+
|     217| 6:06|
|     324| 7:59|
+--------+-----+

+--------+---------+---------+
|train_id|min(time)|max(time)|
+--------+---------+---------+
|     217|     6:06|     6:59|
|     324|     7:59|     9:05|
+--------+---------+---------+

+--------+---------+
|train_id|max(time)|
+--------+---------+
|     217|     6:59|
|     324|     9:05|
+--------+---------+

['train_id', 'max(time)']


### Aggregating the same column twice

In [26]:
from pyspark.sql.functions import min, max, col

expr = [min(col("time")).alias("start"), max(col("time")).alias("end")]
dot_df = df.groupBy("train_id").agg(*expr)
dot_df.show()

query = "SELECT train_id, MIN(time) AS start, MAX(time) AS end FROM table1 GROUP BY train_id"
spark.sql(query).show()

+--------+-----+----+
|train_id|start| end|
+--------+-----+----+
|     217| 6:06|6:59|
|     324| 7:59|9:05|
+--------+-----+----+

+--------+-----+----+
|train_id|start| end|
+--------+-----+----+
|     217| 6:06|6:59|
|     324| 7:59|9:05|
+--------+-----+----+



### Aggregate dot SQL

In [40]:
df = spark.read.csv("trainsched.txt", header=True)
df.createOrReplaceTempView("table1")

spark.sql("""SELECT *, LEAD(time, 1) OVER(PARTITION BY train_id ORDER BY time) AS time_next FROM table1""").show()

from pyspark.sql.functions import lead
from pyspark.sql import Window

df.withColumn("time_next", lead("time", 1).over(Window.partitionBy("train_id").orderBy("time"))).show()

+--------+-------------+----+---------+
|train_id|      station|time|time_next|
+--------+-------------+----+---------+
|     217|       Gilroy|6:06|     6:15|
|     217|   San Martin|6:15|     6:21|
|     217|  Morgan Hill|6:21|     6:36|
|     217| Blossom Hill|6:36|     6:42|
|     217|      Capitol|6:42|     6:50|
|     217|       Tamien|6:50|     6:59|
|     217|     San Jose|6:59|     null|
|     324|San Francisco|7:59|     8:03|
|     324|  22nd Street|8:03|     8:16|
|     324|     Millbrae|8:16|     8:24|
|     324|    Hillsdale|8:24|     8:31|
|     324| Redwood City|8:31|     8:37|
|     324|    Palo Alto|8:37|     9:05|
|     324|     San Jose|9:05|     null|
+--------+-------------+----+---------+

+--------+-------------+----+---------+
|train_id|      station|time|time_next|
+--------+-------------+----+---------+
|     217|       Gilroy|6:06|     6:15|
|     217|   San Martin|6:15|     6:21|
|     217|  Morgan Hill|6:21|     6:36|
|     217| Blossom Hill|6:36|     6:42|

### Convert window function from dot notation to SQL

In [86]:
df = df.withColumn("time_timestamp", df.time.cast("timestamp"))
df.createOrReplaceTempView("table1")
df.show()

+--------+-------------+----+-------------------+
|train_id|      station|time|     time_timestamp|
+--------+-------------+----+-------------------+
|     324|San Francisco|7:59|2021-09-19 07:59:00|
|     324|  22nd Street|8:03|2021-09-19 08:03:00|
|     324|     Millbrae|8:16|2021-09-19 08:16:00|
|     324|    Hillsdale|8:24|2021-09-19 08:24:00|
|     324| Redwood City|8:31|2021-09-19 08:31:00|
|     324|    Palo Alto|8:37|2021-09-19 08:37:00|
|     324|     San Jose|9:05|2021-09-19 09:05:00|
|     217|       Gilroy|6:06|2021-09-19 06:06:00|
|     217|   San Martin|6:15|2021-09-19 06:15:00|
|     217|  Morgan Hill|6:21|2021-09-19 06:21:00|
|     217| Blossom Hill|6:36|2021-09-19 06:36:00|
|     217|      Capitol|6:42|2021-09-19 06:42:00|
|     217|       Tamien|6:50|2021-09-19 06:50:00|
|     217|     San Jose|6:59|2021-09-19 06:59:00|
+--------+-------------+----+-------------------+



In [91]:
window = Window.partitionBy('train_id').orderBy('time')
dot_df = df.withColumn('diff_min', 
                    (unix_timestamp(lead('time', 1).over(window),'H:m') 
                     - unix_timestamp('time', 'H:m'))/60)
dot_df.show()

query = """SELECT *, (UNIX_TIMESTAMP(LEAD(time_timestamp, 1) OVER (PARTITION BY train_id ORDER BY time_timestamp), 'H:m') 
- UNIX_TIMESTAMP(time_timestamp, 'H:m'))/60 AS diff_min FROM table1"""

spark.sql(query).show()

+--------+-------------+----+-------------------+--------+
|train_id|      station|time|     time_timestamp|diff_min|
+--------+-------------+----+-------------------+--------+
|     217|       Gilroy|6:06|2021-09-19 06:06:00|     9.0|
|     217|   San Martin|6:15|2021-09-19 06:15:00|     6.0|
|     217|  Morgan Hill|6:21|2021-09-19 06:21:00|    15.0|
|     217| Blossom Hill|6:36|2021-09-19 06:36:00|     6.0|
|     217|      Capitol|6:42|2021-09-19 06:42:00|     8.0|
|     217|       Tamien|6:50|2021-09-19 06:50:00|     9.0|
|     217|     San Jose|6:59|2021-09-19 06:59:00|    null|
|     324|San Francisco|7:59|2021-09-19 07:59:00|     4.0|
|     324|  22nd Street|8:03|2021-09-19 08:03:00|    13.0|
|     324|     Millbrae|8:16|2021-09-19 08:16:00|     8.0|
|     324|    Hillsdale|8:24|2021-09-19 08:24:00|     7.0|
|     324| Redwood City|8:31|2021-09-19 08:31:00|     6.0|
|     324|    Palo Alto|8:37|2021-09-19 08:37:00|    28.0|
|     324|     San Jose|9:05|2021-09-19 09:05:00|    nul